# Mustererkennung/Machine Learning - Assignment 6



In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

### Load the spam dataset:

In [2]:
data = np.array(pd.read_csv('./spambase/spambase.data', header=None))

X = data[:,:-1] # features
y = data[:,-1] # Last column is label

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True, stratify=y)


In [3]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train[1])
print(y_train[1])

(3450, 57)
(1151, 57)
(3450,)
(1151,)
[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     1.14   0.     0.     0.     0.     0.     0.     2.29   0.
  0.     0.     0.     0.     1.14   1.14   0.     0.     0.     0.
  1.14   0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     2.29   0.     0.     0.     0.     0.     0.
  0.     0.596  0.     0.198  2.133 14.    64.   ]
0.0


In [178]:
def majority(y):
    counts = np.bincount(y)
    return np.argmax(counts)

In [179]:
class Node:
    x_data = None
    y_data = None
    
    left_child = None
    right_child = None
    
    j = 0
    z = 0

    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def set_j_z(self, j, z):
        self.j = j
        self.z = z
        
    def set_right_child(self, right_child):
        self.right_child = right_child
        
    def set_left_child(self, left_child):
        self.left_child = left_child
        
    # use misclassification error
    def loss_function(self):
        zero_pos, zero_neg, one_pos, one_neg = 0, 0, 0, 0
        temp_max = 0
        
        if(self.left_child != None):
            tot_num = self.left_child.y_data.size
            maj = majority(self.left_child.y_data)
            
            for i in self.left_child.y_data:
                if(i != maj):
                    zero_neg += 1
                else:
                    zero_pos += 1
                    
            if(zero_pos > zero_neg):
                temp_max = zero_pos / tot_num
            else:
                temp_max = zero_neg / tot_num
                
        if(self.right_child != None):
            tot_num = self.right_child.y_data.size
            maj = majority(self.right_child.y_data)
            
            for i in self.right_child.y_data:
                if(i != maj):
                    one_neg += 1
                else:
                    one_pos += 1
                    
            if(one_pos > one_neg):
                temp = one_pos / tot_num
            else:
                temp = one_neg / tot_num
                
            if(temp > temp_max):
                return 1 - temp
            else:
                return 1 - temp_max
                
        
node = Node(np.array([1]),np.array([1]))
print(node.set_j_z(1,2))
print(node.z)

None
2


In [193]:
'''
Some helper function
'''
global current


def square(num):
    return pow(num, 2)

def cal_Q_tot(c1, c2, y_1, y_2):
    res = 0
    
    for i in y_1:
        res += square(i - c1)
        
    for i in y_2:
        res += square(i - c2)
        
    return res

# reorder x and y according to j-dimension
def reorder_data(x_data, y_data, j):
    index = x_data[:,j-1].argsort()
    
    temp_x = x_data[index]
    temp_y = y_data[index]
    
    return temp_x, temp_y

class Decision_tree:
    root = None
    
    def __init__(self, root):
        global current
        
        self.root = root
        current = self.root
        
    def inOrder(self, node):
        if node == None:
            return None
        self.inOrder(node.left_child)
        
        print("j = " + str(node.j))
        print("z = " + str(node.z))
        print("x_data: ",end='')
        print(node.x_data)
        print("y_data: ",end='')
        print(node.y_data)
        print("----------------------")
        self.inOrder(node.right_child)
        
    # calculate impurity at x to get j and z for splitting
    def impurity(self, x_data, y_data):
        # number and dimension of datas
        # 2,4
        length = x_data.shape[0]
        dimension = x_data.shape[1]
        
        data_counter = 0
        dim_counter = 1
        cell_in_table = dimension * (length - 1)
        res = np.zeros((cell_in_table, 5))
        
        for i in range(dimension):
            for j in range(length - 1):
                
#                 print("data_counter: " + str(data_counter))
                res[data_counter][0] = i + 1
#                 print(res[data_counter][0])
                
                if j + 1 <= length - 1:
                    z = (x_data[j][i] + x_data[j + 1][i]) / 2.0
#                     if(j == 1):
#                         print("z: " + str(z))
#                         print("j: " + str(j))
#                         print(x_data[j][i])
#                         print(x_data[j+1][i])
                    res[data_counter][1] = z

                    y_1 = y_data[:j + 1]
                    y_2 = y_data[j + 1:]
#                     print("y1 ", end='')
#                     print(y_1)
#                     print(y_2)
#                     c1 = np.average(y_1)
#                     c2 = np.average(y_2)

                    c1 = majority(y_1)
                    c2 = majority(y_2)

                    res[data_counter][2] = c1
                    res[data_counter][3] = c2

                    Q_tot = cal_Q_tot(c1, c2, y_1, y_2)
                    res[data_counter][4] = Q_tot
                
                data_counter += 1
            x_data , y_data = reorder_data(x_data, y_data, i)
            dim_counter += 1
        
        # get index of smallest Q_tot
        y = res[...,-1]
        
        if y.size != 0:
            index = np.argmin(y)
        else:
            return None, None
        # return j and z 
        # self.root.set_j_z(res[index][0], res[index][1])
        print(res)
        
        return res[index][0], res[index][1]
    
    # split node x into 2 branches
    def split_x(self, x_data, y_data, j, z):
        if(j == None):
            return
        j = int(j)
        j_d_x_data = x_data[...,j - 1]
        
        # get all indices
        indices_le_than_x = np.where(j_d_x_data <= z)
        indices_gr_than_x = np.where(j_d_x_data > z)
        
        le_than_x = x_data[indices_le_than_x]
        gr_than_x = x_data[indices_gr_than_x]
        
        y_1 = y_data[indices_le_than_x]
        y_2 = y_data[indices_gr_than_x]
        
        return le_than_x, gr_than_x, y_1, y_2
    
    # check if leaves contain just one element
    def check_child_data_size(self):
        if(self.root.left_child.shape[0] == 1 and self.root.right_child.shape[0] == 1):
            return True
        return False
    
    def insert_left(self, left_child, current):
        current.set_left_child(left_child)
        
    def insert_right(self, right_child, current):
        current.set_right_child(right_child)
    
    def generate_decision_tree(self, temp):
        counter = 0
        
        if(temp != None):

                temp_x = temp.x_data
                temp_y = temp.y_data

                j, z = self.impurity(temp_x, temp_y)
                
                temp.set_j_z(j, z)

                if(self.split_x(temp_x, temp_y, j, z) != None):
                    le_than_x, gr_than_x, y_1, y_2 = self.split_x(temp_x, temp_y, j, z)

                    left = Node(le_than_x, y_1)
                    right = Node(gr_than_x, y_2)

                    self.insert_left(left, temp)
                    self.insert_right(right, temp)

                    counter += 1

                    self.generate_decision_tree(temp.left_child)
                    self.generate_decision_tree(temp.right_child)
                else:
                    return
              

In [194]:
#tree = Decision_tree(Node(X_train, y_test.astype(np.int64)))
tree = Decision_tree(Node(np.array([[1,1],[2,3],[3,7],[4,5]]), np.array([1,1,0,1])))
#j, z = tree.impurity(np.array([[1,1],[2,3],[3,7],[4,5]]), np.array([1,1,0,1]))
#print(tree.impurity(np.array([[1,1],[2,3],[3,7],[4,5]]), np.array([1,1,0,1])))
#print(tree.impurity(np.array([[1,1],[2,3],[3,7],[4,5]]), np.array([3,5,6,2.5])))
#print(tree.split_x(np.array([[1,1],[2,3],[3,7],[4,5]]), np.array([1,1,0,1]), j, z))

tree.generate_decision_tree(tree.root)
# print("left_child ", end='')
#print(tree.root.left_child.x_data)
tree.inOrder(tree.root)
#print(tree.root.j)

[[1.  1.5 1.  1.  1. ]
 [1.  2.5 1.  0.  1. ]
 [1.  3.5 1.  1.  1. ]
 [2.  2.  1.  1.  1. ]
 [2.  4.  1.  0.  1. ]
 [2.  6.  1.  0.  0. ]]
[[1.  1.5 1.  1.  0. ]
 [1.  3.  1.  1.  0. ]
 [2.  2.  1.  1.  0. ]
 [2.  4.  1.  1.  0. ]]
[[1. 3. 1. 1. 0.]
 [2. 4. 1. 1. 0.]]
j = None
z = None
x_data: [[1 1]]
y_data: [1]
----------------------
j = 1.0
z = 1.5
x_data: [[1 1]
 [2 3]
 [4 5]]
y_data: [1 1 1]
----------------------
j = None
z = None
x_data: [[2 3]]
y_data: [1]
----------------------
j = 1.0
z = 3.0
x_data: [[2 3]
 [4 5]]
y_data: [1 1]
----------------------
j = None
z = None
x_data: [[4 5]]
y_data: [1]
----------------------
j = 2.0
z = 6.0
x_data: [[1 1]
 [2 3]
 [3 7]
 [4 5]]
y_data: [1 1 0 1]
----------------------
j = None
z = None
x_data: [[3 7]]
y_data: [0]
----------------------


In [210]:
a = np.array([1,1,0,0])
counts = np.bincount(a)
print(counts)
print(np.argmax(counts))
# print(np.average(a))
# print(a[5:])
# b = np.array([1,2,3])
# print(a[b])

[2 2]
0


In [139]:
temp = np.array([[1,2,3],[4,5,6]])
i = 1
#temp[i][1] = 2
print(temp.size)

6


In [86]:
arr = [[1,3,2],[10,5,6],[7,9,8]]

x = np.argsort(arr, axis= 1)
print(x)
#print(arr[x])

[[0 2 1]
 [1 2 0]
 [0 2 1]]


In [88]:
# a = [[1,3,2],[10,5,6],[7,9,8]] # First column
# b = [1,2,3] # Second column
print(reorder_data(np.array([[1,1],[2,3],[3,7],[4,5]]),np.array([1,1,0,1]), 1)) # Sort by a, then by b


(array([[1, 1],
       [2, 3],
       [4, 5],
       [3, 7]]), array([1, 1, 1, 0]))


In [139]:
a = np.array([[1,1],[2,3],[3,7],[4,5]])
b = np.array([1,2,3,4])
print(a[a[:,1].argsort()])
print(a.size)

[[1 1]
 [2 3]
 [4 5]
 [3 7]]
8
